# Ranking des concepts par LIG-score

---
Goal of the notebook: Ranker des concepts par LIG-score.

Inputs of the notebook:
-.
Output of the notebook:

-.
Takeaways: 
- .
- .

In [1]:
# !pip install -r requirements.txt

In [2]:
import os
import sys
sys.path.append('../../run_experiments/')
sys.path.append('../../run_experiments/scripts')
sys.path.append('../../run_experiments/models')
sys.path.append('../../run_experiments/data')

import time
from tqdm import tqdm
import numpy as np
import pandas as pd
import json


# import fonction for getting PLM and tokenizer
from models.utils import load_model_and_tokenizer

# library for managing memory RAM

import pickle
import json

import pandas as pd
import pickle, os, json, gc

/home/bhan/.conda/envs/good_env/lib/python3.12/site-packages/sklearn/__init__.py:82: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  import scipy.linalg  # noqa


In [3]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

20

# 0. autoreload

In [4]:
#code for autoreload script associated with jupyter notebook
%load_ext autoreload
%autoreload 2

# 1.SETUP	 ENVIRONMENT VARIABLES

In [5]:
# import config
from load_config import load_config

model_name = 'bert-base-uncased'    # 'bert-base-uncased' ou 'deberta-large' or 'gemma'
dataset    = 'agnews'               # 'movies' / 'agnews' / 'dbpedia' / 'medical'/ 'ledgar'/ n24news
annotation = 'C3M'       # 'C3M' ou 'our_annotation' ou 'combined_annotation'
config = load_config(model_name, dataset)
config.annotation = annotation

agg_mode = 'abs'
agg_scope ="all"

# 2. Black Box Model

In [6]:
from models.BaselineModel import BaselineModel

In [7]:
# import the PLM model and tokenizer and bottleneck layer
embedder_model, embedder_tokenizer, _, classifier = load_model_and_tokenizer(config, n_concepts = 4)

black_box_model = BaselineModel(embedder_model, classifier, None, None, None, config, save_path = config.SAVE_PATH)
black_box_model.load_model()

# Fonction pour vérifier si les paramètres du modèle sont en float64
def check_parameters_dtype(model):
    for param in model.parameters():
        if param.dtype != torch.float64:
            return False
    return True
    
# Fonction pour convertir les paramètres du modèle en float64
def convert_parameters_to_float64(model):
    for param in model.parameters():
        param.data = param.data.double()

# Supposons que `black_box_model` est l'instance du modèle
convert_parameters_to_float64(black_box_model)

# Vérifier si tous les paramètres sont maintenant en float64
is_float64 = check_parameters_dtype(black_box_model)

print(f"Tous les paramètres sont maintenant en float64 : {is_float64}")

/home/bhan/Yann_CBM/Launch/notebooks/../run_experiments/models/BaselineModel.py:199: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.classifier.load_state_dict(torch.load

Aucune performance enregistrée trouvée.
Tous les paramètres sont maintenant en float64 : True


## 3.0 create dataloader with augmented concepts bellow

In [9]:
# # import the dataframe
from prepare_data import prepare_data_from_csv
df_aug_train, df_aug_test = prepare_data_from_csv(annotation  = config.annotation, config = config, return_test = True)

# dataloaders
from concepts_bank_utils import create_dataloader
notre_loader_train = create_dataloader(df_aug_train, embedder_tokenizer, config.max_len, config.batch_size)
notre_loader_test = create_dataloader(df_aug_test, embedder_tokenizer, config.max_len, config.batch_size)

Chargement des vecteurs CAV

In [10]:
if annotation == 'C3M' :
    file_path = f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs/{config.cavs_type}/cavs_{config.cavs_type}_C3M.json"
elif annotation == 'our_annotation':
    file_path = f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs/{config.cavs_type}/cavs_{config.cavs_type}_our_annotation.json"
else:
    print(f'no cavs computed for annotation : {annotation}')

In [11]:
# Chargement des vecteurs CAV à partir d'un fichier JSON et conversion sur GPU
if os.path.exists(file_path):
    with open(file_path, 'r') as f:
        cavs_vectors = json.load(f)
    cavs = {k: torch.tensor(v, dtype=torch.float32).to(config.device) for k, v in cavs_vectors.items()}
    print("cavs chargés à partir de", file_path)
else:
    print("Fichier cavs introuvable :", file_path)


cavs chargés à partir de /home/bhan/Yann_CBM/Launch/dbfs/results_agnews//blue_checkpoints/bert-base-uncased/cavs/mean/cavs_mean_C3M.json


importer les vectors CAVS selon le type "mean" ou "svm"

# 1.Calculer le concept identifiability score

In [12]:
if config.model_name =='gemma':
    from new_heuristique import compute_cosine_matrix_and_metrics_gemma_version as compute_cosine_matrix_and_metrics
else:
    from new_heuristique import compute_cosine_matrix_and_metrics

concept identifiability score = F1 score de detection sur le validation set

prendre le threshold en se basant sur la mediane cosine similarity sur le train set (pour l'utiliser comme threshold sur le test)

In [13]:
# cosine_df, metrics, well_detected_concepts = compute_cosine_matrix_and_metrics(
cosine_train, cosine_val, cosine_df, thresholds, metrics, well_detected_concepts = compute_cosine_matrix_and_metrics(
    df=df_aug_train,
    text_column="text",
    embedder_model=embedder_model,
    embedder_tokenizer=embedder_tokenizer,
    cavs=cavs,
    f1_cutoff=0.1,  # Par exemple, ne retenir que les concepts dont le F1 >= 0.5
    device=config.device,  # ex: torch.device("cuda")
    save_dir=f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs/{config.cavs_type}",
    config = config,
    annotation = annotation,
    cos_cubed = False
)

Chargement de cosine_df depuis /home/bhan/Yann_CBM/Launch/dbfs/results_agnews//blue_checkpoints/bert-base-uncased/cavs/mean/cosine_df_C3M.pkl


In [14]:
from new_heuristique import (clean_concept_name, rename_dataframe_columns, 
filter_concepts_by_coverage, 
filter_concepts_by_coverage,
order_well_detected_concepts )

# 2.Compute the Global indice for ranking concepts

#### 2.1 load all the 3 metrics

In [17]:
# frequencies or presence_scores
presence_scores = {
    clean_concept_name(concept): df_aug_train[concept].mean()
    for concept in df_aug_train.columns
    if concept not in ['text','label','Unnamed: 0']
        }


In [18]:
# concept identifiability score
file_path_1 = (
f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs/{config.cavs_type}/"
f"detection_concept_{config.cavs_type}_{config.annotation}_{config.agg_mode}_{config.agg_scope}.json"
)


if os.path.exists(file_path_1):
    with open(file_path_1, "r") as f:
        well_detected_concepts = json.load(f)
    print(f"Fichier {file_path_1} déjà présent, chargement...")
well_detected_concepts = {key: perf['F1'] for key, perf in well_detected_concepts.items()}


Fichier /home/bhan/Yann_CBM/Launch/dbfs/results_agnews//blue_checkpoints/bert-base-uncased/cavs/mean/detection_concept_mean_C3M_abs_all.json déjà présent, chargement...


In [19]:
### tcavs
file_path_2 = (
    f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs/{config.cavs_type}/"
    f"sorted_macro_concepts_C3M.json"
)
if os.path.exists(file_path_2):
    with open(file_path_2, "r") as f:
        all_concepts_sorted_by_tcavs = json.load(f)
    print(f"Fichier {file_path_2} déjà présent, chargement...")
all_concepts_sorted_by_tcavs = { tuple[0]: tuple[1] for tuple in all_concepts_sorted_by_tcavs}


Fichier /home/bhan/Yann_CBM/Launch/dbfs/results_agnews//blue_checkpoints/bert-base-uncased/cavs/mean/sorted_macro_concepts_C3M.json déjà présent, chargement...


In [20]:
# LIG Ranking 
# --- Chargement et préparation des concepts depuis le fichier JSON ---
file_path_3 = (
    f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs/"
    f"{config.cavs_type}/sorted_macro_concepts_cosine_sm_{config.cavs_type}_{config.annotation}_{config.agg_mode}_{config.agg_scope}.json"
    )
# Charger le fichier JSON
with open(file_path_3, "r") as f:
    sorted_macro_concepts_cosine_sm = json.load(f)

# Supprimer les doublons
sorted_macro_concepts_cosine_sm = list(dict.fromkeys(tuple(item) for item in sorted_macro_concepts_cosine_sm))

# Appliquer le nettoyage sur la liste chargée
all_concepts_sorted_by_lig = [(clean_concept_name(name), score) for name, score in sorted_macro_concepts_cosine_sm]
all_concepts_sorted_by_lig = { tuple[0]: tuple[1] for tuple in all_concepts_sorted_by_lig}


#### 2.2 Combine

In [ ]:
import pandas as pd

# Exemple de données ; à adapter avec vos variables
# presence_scores, well_detected_concepts, all_concepts_sorted_by_tcavs et all_concepts_sorted_by_lig

plot_df = pd.DataFrame({
    "Presence (Mean)": presence_scores,
    "F1 Score detected": well_detected_concepts,
    "TCAVS score": all_concepts_sorted_by_tcavs,
    "LIG score": all_concepts_sorted_by_lig
})

print("Shape du DataFrame initial :", plot_df.shape)

# Normalisation min-max pour les scores LIG 
#(don't automatically normalise TCAVS score: don't do it if blackboxe model is just a linear layer wwithout relu or sigmoid activation function) 
# plot_df["TCAVS score norm"] = (plot_df["TCAVS score"] - plot_df["TCAVS score"].min()) / (plot_df["TCAVS score"].max() - plot_df["TCAVS score"].min())
plot_df["LIG score norm"] = (plot_df["LIG score"] - plot_df["LIG score"].min()) / (plot_df["LIG score"].max() - plot_df["LIG score"].min())


plot_df["combined_score_TCAVS"] =  plot_df["F1 Score detected"] * plot_df["TCAVS score"]
plot_df["combined_score_LIG"] = plot_df["F1 Score detected"] * plot_df["LIG score norm"]

# Calcul des scores combinés en utilisant les scores normalisés
# plot_df["combined_score_TCAVS"] = plot_df["Presence (Mean)"] * plot_df["F1 Score detected"] * plot_df["TCAVS score"]
# plot_df["combined_score_LIG"] = plot_df["Presence (Mean)"] * plot_df["F1 Score detected"] * plot_df["LIG score norm"]



Now save the score for ulterior usage

In [23]:

# save the score
path_plot_df = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/combined_score_concept_{config.annotation}_{config.agg_mode}_{config.agg_scope}.csv"
    )
plot_df['concept'] = plot_df.index
plot_df.to_csv(path_plot_df, index = False)
plot_df.head(5)

Shape du DataFrame initial : (41, 4)


,Presence (Mean),F1 Score detected,TCAVS score,LIG score,LIG score norm,combined_score_TCAVS,combined_score_LIG,concept
international events,0.46525,0.757895,0.25,0.029520,0.806204,0.189474,0.611018,international events
global politics,0.31275,0.719524,0.25,0.021578,0.149151,0.179881,0.107318,global politics
international relations,0.29100,0.688822,0.25,0.021659,0.155840,0.172205,0.107346,international relations
foreign affairs,0.29325,0.710448,0.25,0.021693,0.158712,0.177612,0.112757,foreign affairs
"News about wars, conflicts",0.19500,0.512224,0.25,0.021624,0.153000,0.128056,0.078370,"News about wars, conflicts"


In [ ]:
path_plot_df = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/combined_score_concept_{config.annotation}_{config.agg_mode}_{config.agg_scope}.csv"
    )
plot_df = pd.read_csv(path_plot_df)

df_sorted_tcavs = plot_df.sort_values(by ='combined_score_TCAVS', ascending = False)
df_sorted_lig = plot_df.sort_values(by ='combined_score_LIG', ascending = False)
combined_score_TCAVS = dict(zip(df_sorted_tcavs['concept'], df_sorted_tcavs['combined_score_TCAVS']))
combined_score_LIG = dict(zip(df_sorted_lig['concept'], df_sorted_lig['combined_score_LIG']))

path_combined_score_LIG = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/combined_score_LIG_{config.annotation}_{config.agg_mode}_{config.agg_scope}.json"
    )

path_combined_score_TCAVS = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/combined_score_TCAVS_{config.annotation}_{config.agg_mode}_{config.agg_scope}.json"
    )

with open(path_combined_score_LIG, 'w') as f:
    json.dump(combined_score_LIG, f, ensure_ascii=False, indent=4)

with open(path_combined_score_TCAVS, 'w') as f:
    json.dump(combined_score_TCAVS, f, ensure_ascii=False, indent=4)



# BY TCAVS 

In [ ]:
import json
import re
import matplotlib.pyplot as plt

# --- Partie 2 : Récupération de la liste ordonnée des concepts et calcul du coverage cumulatif ---

plot_df = plot_df.set_index('concept', drop=False)
ordered_concepts = plot_df.sort_values(by="combined_score_TCAVS", ascending=False).index.tolist()

# Fonction de nettoyage identique à celle utilisée pour les concepts
def clean_concept_name(name):
    name = name.replace("cos_", "").replace("dummy_", "").strip()
    name = re.sub(r"Let me know.*", "", name)
    return " ".join(name.split())

# On s'assure que les noms de colonnes du DataFrame sont nettoyés pour correspondre aux noms présents dans ordered_concepts
df_aug_train.rename(columns=lambda x: clean_concept_name(x), inplace=True)

# --- Calcul de la courbe cumulative à partir de df_aug_train ---
counts = []
concepts_iter = []
for concept in ordered_concepts:
    # Ajout du concept courant dans la liste cumulée
    concepts_iter.append(concept)
    filtered_data = df_aug_train[concepts_iter][df_aug_train[concepts_iter].eq(1).any(axis=1)]
    counts.append(len(filtered_data))

# Calcul du pourcentage de lignes couvertes par rapport au total du DataFrame
percentage = [100 * c / df_aug_train.shape[0] for c in counts]

# Positions pour l'axe des x (une par concept)
positions = np.arange(len(ordered_concepts))

# --- Affichage du graphique combiné ---
fig, ax1 = plt.subplots(figsize=(18, 15))

values = plot_df.loc[ordered_concepts, "combined_score_TCAVS"]  # ou "LIG score", selon le choix

ax1.bar(positions, values, color='skyblue', alpha=0.7, label="Valeur moyenne des concepts")
ax1.set_ylabel("combined_score_TCAVS")
ax1.set_xlabel("Concepts")
ax1.set_title("Valeur moyenne des concepts et couverture cumulative")
ax1.set_xticks(positions)
ax1.set_xticklabels(ordered_concepts, rotation=45, ha='right')
ax1.grid(axis="y", linestyle="--", alpha=0.7)

# Axe secondaire : Courbe cumulative (pourcentage de lignes couvertes)
ax2 = ax1.twinx()
ax2.plot(positions, percentage, marker='o', color='darkorange', label="Couverture cumulative")
ax2.set_ylabel("Pourcentage de lignes avec au moins un concept (%)", color='darkorange')
ax2.tick_params(axis='y', colors='darkorange')
ax2.grid(axis="y", linestyle="--", alpha=0.5)

# Légendes combinées
ax1.legend(loc="upper left")
ax2.legend(loc="upper right")

plt.tight_layout()
plt.show()


In [ ]:
path_coverage = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/sorted_macro_concepts_coverage_{config.annotation}_{config.agg_mode}_{config.agg_scope}_TCAVS.json"
    )

score_dict = dict(zip(ordered_concepts, percentage))

with open(path_coverage, 'w') as f:
    json.dump(score_dict, f, ensure_ascii=False, indent=4)




In [ ]:
# Metrics d'aggregation par cluster

path_plot_df = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/combined_score_concept_{config.annotation}_{config.agg_mode}_{config.agg_scope}.csv"
    )

plot_df = pd.read_csv(path_plot_df)
# plot_df


In [ ]:
path_cluster_dict_strat_reassignation = f"{config.SAVE_PATH_CONCEPTS}/cluster_dict_strat_reassignation_C3M.json"

path_cluster_dict_strat_noise_cluster = f"{config.SAVE_PATH_CONCEPTS}/cluster_dict_strat_noise_cluster_C3M.json"
                                    
with open(path_cluster_dict_strat_reassignation, 'r') as f:
   cluster_dict_strat_reassignation = json.load(f)

with open(path_cluster_dict_strat_noise_cluster, 'r') as f:
    cluster_dict_strat_noise_cluster = json.load(f)


def new_columns_reassignation(concept):
    for k, v in cluster_dict_strat_reassignation.items():
        if concept in v:
            return int(k)
    return -1  # ou autre valeur par défaut si pas trouvé

def new_columns_noisy(concept):
    for k, v in path_cluster_dict_strat_noise_cluster.items():
        if concept in v:
            return int(k)
    return -1  # ou autre valeur par défaut si pas trouvé

plot_df['cluster'] = plot_df['concept'].apply(new_columns_reassignation)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
import hdbscan
from umap import UMAP

# 0) On met le nom du concept en index
plot_df = plot_df.set_index("concept")

# 4) Construction : cluster → concepts triés par score
cluster_groups = {}
for c in sorted(set(set(plot_df['cluster'].to_list()))):
    lst = (plot_df
           .loc[plot_df["cluster"] == c]
           .sort_values("combined_score_TCAVS", ascending=False)
           .index
           .tolist())
    cluster_groups[c] = lst

# 5) Sélection round-robin + suivi du coverage par étape
total = df_aug_train.shape[0]
working = {c: cluster_groups[c].copy() for c in cluster_groups}

# Stockage des résultats
coverage_evolution = []
selected_concepts = []

while any(working[c] for c in working):
    current_selection = []
    for c in sorted(working):
        if working[c]:
            concept = working[c].pop(0)
            current_selection.append(concept)
            selected_concepts.append(concept)
    
    # Calcul du coverage après avoir ajouté ces concepts
    mask = df_aug_train[selected_concepts].eq(1).any(axis=1)
    coverage = mask.sum() * 100 / total
    
    coverage_evolution.append((current_selection.copy(), coverage))

# 6) Affichage
for idx, (concepts, cov) in enumerate(coverage_evolution):
    print(f"Step {idx+1}: Concepts {concepts} → Coverage {cov:.2f}%")



In [ ]:
import matplotlib.pyplot as plt

# Cumuler le nombre total de concepts sélectionnés à chaque étape
n_concepts_selected = np.cumsum([len(concepts) for concepts, _ in coverage_evolution])
coverage_values = [cov for _, cov in coverage_evolution]

# Tracé
plt.figure(figsize=(12, 6))
plt.plot(n_concepts_selected, coverage_values, marker='o', linestyle='-')
plt.xlabel("Nombre de concepts sélectionnés")
plt.ylabel("Coverage cumulé (%)")
plt.title("Évolution du coverage en sélection round-robin par cluster")
plt.grid(True)
plt.show()


In [ ]:
import pickle
path_coverage = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/sorted_macro_concepts_coverage_MJ_{config.annotation}_{config.agg_mode}_{config.agg_scope}_TCAVS.pkl"
    )
# Sauvegarde
with open(path_coverage, "wb") as f:
    pickle.dump(coverage_evolution, f)

print(f"✅ Fichier {path_coverage} en .pkl' sauvegardé !")


In [ ]:
with open(path_coverage, "rb") as f:
    loaded_coverage_evolution_ = pickle.load(f)
loaded_coverage_evolution_

# BY LIG

In [ ]:
# Metrics d'aggregation par cluster

path_plot_df = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/combined_score_concept_{config.annotation}_{config.agg_mode}_{config.agg_scope}.csv"
    )

plot_df = pd.read_csv(path_plot_df)
# plot_df


In [ ]:
import json
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



# --- Partie 2 : Récupération de la liste ordonnée des concepts et calcul du coverage cumulatif ---

# Ici, nous choisissons par exemple de trier les concepts selon le combined_score_TCAVS décroissant.
# La même démarche peut être faite avec combined_score_LIG

plot_df = plot_df.set_index('concept', drop=False)
# ordered_concepts = plot_df.sort_values(by="combined_score_LIG", ascending=False).to_list()
ordered_concepts = plot_df.sort_values(by="combined_score_LIG", ascending=False)['concept'].to_list()

print("\nListe des concepts triée par combined_score_TCAVS (du plus grand au plus petit) :")
print(ordered_concepts)


# --- Chargement du DataFrame de données réelles (df_aug_train) et préparation des colonnes ---
# Exemple de chargement : on suppose que df_aug_train contient une colonne pour chacun des concepts,
# avec 1 indiquant la présence du concept dans la ligne.
# Attention : vous devez adapter le chemin et le nettoyage des colonnes afin que les noms correspondent.

# Fonction de nettoyage identique à celle utilisée pour les concepts
def clean_concept_name(name):
    name = name.replace("cos_", "").replace("dummy_", "").strip()
    name = re.sub(r"Let me know.*", "", name)
    return " ".join(name.split())

# On s'assure que les noms de colonnes du DataFrame sont nettoyés pour correspondre aux noms présents dans ordered_concepts
df_aug_train.rename(columns=lambda x: clean_concept_name(x), inplace=True)

# --- Calcul de la courbe cumulative à partir de df_aug_train ---
counts = []
concepts_iter = []
for concept in ordered_concepts:
    # Ajout du concept courant dans la liste cumulée
    concepts_iter.append(concept)
    # Sélection des lignes où au moins un des concepts cumulés est présent (valeur == 1)
    # On suppose que dans df_aug_train, pour chaque concept, la présence est indiquée par 1 (sinon adapter)
    filtered_data = df_aug_train[concepts_iter][df_aug_train[concepts_iter].eq(1).any(axis=1)]
    counts.append(len(filtered_data))

# Calcul du pourcentage de lignes couvertes par rapport au total du DataFrame
percentage = [100 * c / df_aug_train.shape[0] for c in counts]

# Positions pour l'axe des x (une par concept)
positions = np.arange(len(ordered_concepts))

# --- Affichage du graphique combiné ---
fig, ax1 = plt.subplots(figsize=(18, 15))

# Graphique 1 : Barres verticales pour la valeur moyenne (ici, on peut choisir d'afficher la valeur du score associé du JSON)
# Dans cet exemple, on affiche les scores moyens provenant du JSON (extrait du plot_df précédemment)
# Si nécessaire, vous pouvez adapter la barre pour afficher une autre valeur.
values = plot_df.loc[ordered_concepts, "combined_score_LIG"]  # ou "LIG score", selon le choix

ax1.bar(positions, values, color='skyblue', alpha=0.7, label="Valeur moyenne des concepts")
ax1.set_ylabel("combined_score_LIG")
ax1.set_xlabel("Concepts")
ax1.set_title("Valeur moyenne des concepts et couverture cumulative")
ax1.set_xticks(positions)
ax1.set_xticklabels(ordered_concepts, rotation=45, ha='right')
ax1.grid(axis="y", linestyle="--", alpha=0.7)

# Axe secondaire : Courbe cumulative (pourcentage de lignes couvertes)
ax2 = ax1.twinx()
ax2.plot(positions, percentage, marker='o', color='darkorange', label="Couverture cumulative")
ax2.set_ylabel("Pourcentage de lignes avec au moins un concept (%)", color='darkorange')
ax2.tick_params(axis='y', colors='darkorange')
ax2.grid(axis="y", linestyle="--", alpha=0.5)

# Légendes combinées
ax1.legend(loc="upper left")
ax2.legend(loc="upper right")

plt.tight_layout()
plt.show()


In [ ]:
path_coverage = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/sorted_macro_concepts_coverage_{config.annotation}_{config.agg_mode}_{config.agg_scope}.json"
    )

score_dict = dict(zip(ordered_concepts, percentage))

with open(path_coverage, 'w') as f:
    json.dump(score_dict, f, ensure_ascii=False, indent=4)




In [ ]:
# Metrics d'aggregation par cluster

path_plot_df = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/combined_score_concept_{config.annotation}_{config.agg_mode}_{config.agg_scope}.csv"
    )

plot_df = pd.read_csv(path_plot_df)
# plot_df


In [ ]:
path_cluster_dict_strat_reassignation = f"{config.SAVE_PATH_CONCEPTS}/cluster_dict_strat_reassignation_C3M.json"

path_cluster_dict_strat_noise_cluster = f"{config.SAVE_PATH_CONCEPTS}/cluster_dict_strat_noise_cluster_C3M.json"
                                    
with open(path_cluster_dict_strat_reassignation, 'r') as f:
   cluster_dict_strat_reassignation = json.load(f)

with open(path_cluster_dict_strat_noise_cluster, 'r') as f:
    cluster_dict_strat_noise_cluster = json.load(f)

In [ ]:
def new_columns_reassignation(concept):
    for k, v in cluster_dict_strat_reassignation.items():
        if concept in v:
            return int(k)
    return -1  # ou autre valeur par défaut si pas trouvé

def new_columns_noisy(concept):
    for k, v in path_cluster_dict_strat_noise_cluster.items():
        if concept in v:
            return int(k)
    return -1  # ou autre valeur par défaut si pas trouvé

plot_df['cluster'] = plot_df['concept'].apply(new_columns_reassignation)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
import hdbscan
from umap import UMAP

# 0) On met le nom du concept en index
plot_df = plot_df.set_index("concept")

# 4) Construction : cluster → concepts triés par score
cluster_groups = {}
for c in sorted(set(set(plot_df['cluster'].to_list()))):
    lst = (plot_df
           .loc[plot_df["cluster"] == c]
           .sort_values("combined_score_TCAVS", ascending=False)
           .index
           .tolist())
    cluster_groups[c] = lst

# 5) Sélection round-robin + suivi du coverage par étape
total = df_aug_train.shape[0]
working = {c: cluster_groups[c].copy() for c in cluster_groups}

# Stockage des résultats
coverage_evolution = []
selected_concepts = []

while any(working[c] for c in working):
    current_selection = []
    for c in sorted(working):
        if working[c]:
            concept = working[c].pop(0)
            current_selection.append(concept)
            selected_concepts.append(concept)
    
    # Calcul du coverage après avoir ajouté ces concepts
    mask = df_aug_train[selected_concepts].eq(1).any(axis=1)
    coverage = mask.sum() * 100 / total
    
    coverage_evolution.append((current_selection.copy(), coverage))

# 6) Affichage
for idx, (concepts, cov) in enumerate(coverage_evolution):
    print(f"Step {idx+1}: Concepts {concepts} → Coverage {cov:.2f}%")



In [ ]:
import matplotlib.pyplot as plt

# Cumuler le nombre total de concepts sélectionnés à chaque étape
n_concepts_selected = np.cumsum([len(concepts) for concepts, _ in coverage_evolution])
coverage_values = [cov for _, cov in coverage_evolution]

# Tracé
plt.figure(figsize=(12, 6))
plt.plot(n_concepts_selected, coverage_values, marker='o', linestyle='-')
plt.xlabel("Nombre de concepts sélectionnés")
plt.ylabel("Coverage cumulé (%)")
plt.title("Évolution du coverage en sélection round-robin par cluster")
plt.grid(True)
plt.show()


In [ ]:
import pickle
path_coverage = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/sorted_macro_concepts_coverage_MJ_{config.annotation}_{config.agg_mode}_{config.agg_scope}.pkl"
    )


# Sauvegarde
with open(path_coverage, "wb") as f:
    pickle.dump(coverage_evolution, f)

print("✅ Fichier 'coverage_evolution.pkl' sauvegardé !")


In [ ]:
with open(path_coverage, "rb") as f:
    loaded_coverage_evolution = pickle.load(f)
loaded_coverage_evolution

# Abblation study on 10 random generation

In [ ]:
# Metrics d'aggregation par cluster

path_plot_df = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/combined_score_concept_{config.annotation}_{config.agg_mode}_{config.agg_scope}.csv"
    )

plot_df = pd.read_csv(path_plot_df)
# plot_df


In [ ]:
plot_df

In [ ]:
import pickle
import json
import pandas as pd
import numpy as np
from pathlib import Path

# On reprend les mêmes df et dicts que précédemment :
# - plot_df : DataFrame chargé depuis combined_score_concept_…csv
# - df_aug_train : ton DataFrame d'entraînement binaire (colonnes « concept » = 1 ou 0)
# - cluster_dict_strat_reassignation et cluster_dict_strat_noise_cluster déjà chargés

# Chemins de base
path_plot_df = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/combined_score_concept_{config.annotation}_{config.agg_mode}_{config.agg_scope}.csv"
    )

# Préparer les clusters
with open(f"{config.SAVE_PATH_CONCEPTS}/cluster_dict_strat_reassignation_C3M.json", "r") as f:
    cluster_reassign = json.load(f)
with open(f"{config.SAVE_PATH_CONCEPTS}/cluster_dict_strat_noise_cluster_C3M.json", "r") as f:
    cluster_noise = json.load(f)

def assign_cluster(concept, dct):
    for k,v in dct.items():
        if concept in v:
            return int(k)
    return -1

plot_df["cluster"] = plot_df.index.map(lambda c: assign_cluster(c, cluster_reassign))

# Nombre total de lignes pourcentage
total = df_aug_train.shape[0]

for run in range(10):
    # 1) Flipper / mélanger les scores LIG
    shuffled = plot_df["combined_score_LIG"].sample(frac=1, random_state=run).values
    plot_df["combined_score_LIG_shuffled"] = shuffled
    
    # 2) Construire les groupes par cluster, triés par score mélangé
    cluster_groups = {
        c: plot_df.loc[plot_df.cluster == c]
                  .sort_values("combined_score_LIG_shuffled", ascending=False)
                  .index
                  .tolist()
        for c in plot_df["cluster"].unique()
    }
    working = {c: lst.copy() for c,lst in cluster_groups.items()}

    # 3) Round‑robin et calcul du coverage cumulatif
    selected = []
    coverage_evolution = []
    while any(working[c] for c in working):
        batch = []
        for c in sorted(working):
            if working[c]:
                batch.append(working[c].pop(0))
                selected.append(batch[-1])
        # coverage après ajout de batch
        mask = df_aug_train[selected].eq(1).any(axis=1)
        coverage = mask.sum() * 100.0 / total
        coverage_evolution.append((batch.copy(), coverage))

    # 4) Sauvegarde du .pkl pour ce run
    out_path = base_cavs / f"sorted_macro_concepts_coverage_MJ_{config.annotation}_{config.agg_mode}_{config.agg_scope}_{run}.pkl"
    with open(out_path, "wb") as f:
        pickle.dump(coverage_evolution, f)
    print(f"✅ run {run} → {out_path.name} ({len(coverage_evolution)} étapes) sauvegardé.")


# Abblation study (SCORE = LIG) without F1

In [ ]:
# Metrics d'aggregation par cluster

path_plot_df = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/combined_score_concept_{config.annotation}_{config.agg_mode}_{config.agg_scope}.csv"
    )

plot_df = pd.read_csv(path_plot_df)
# plot_df


In [ ]:
import json
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



# --- Partie 2 : Récupération de la liste ordonnée des concepts et calcul du coverage cumulatif ---

# Ici, nous choisissons par exemple de trier les concepts selon le combined_score_TCAVS décroissant.
# La même démarche peut être faite avec combined_score_LIG

plot_df = plot_df.set_index('concept', drop=False)
# ordered_concepts = plot_df.sort_values(by="combined_score_LIG", ascending=False).to_list()
ordered_concepts = plot_df.sort_values(by="LIG score", ascending=False)['concept'].to_list()

print("\nListe des concepts triée par le score LIG :")
print(ordered_concepts)


# --- Chargement du DataFrame de données réelles (df_aug_train) et préparation des colonnes ---
# Exemple de chargement : on suppose que df_aug_train contient une colonne pour chacun des concepts,
# avec 1 indiquant la présence du concept dans la ligne.
# Attention : vous devez adapter le chemin et le nettoyage des colonnes afin que les noms correspondent.

# Fonction de nettoyage identique à celle utilisée pour les concepts
def clean_concept_name(name):
    name = name.replace("cos_", "").replace("dummy_", "").strip()
    name = re.sub(r"Let me know.*", "", name)
    return " ".join(name.split())

# On s'assure que les noms de colonnes du DataFrame sont nettoyés pour correspondre aux noms présents dans ordered_concepts
df_aug_train.rename(columns=lambda x: clean_concept_name(x), inplace=True)

# --- Calcul de la courbe cumulative à partir de df_aug_train ---
counts = []
concepts_iter = []
for concept in ordered_concepts:
    # Ajout du concept courant dans la liste cumulée
    concepts_iter.append(concept)
    # Sélection des lignes où au moins un des concepts cumulés est présent (valeur == 1)
    # On suppose que dans df_aug_train, pour chaque concept, la présence est indiquée par 1 (sinon adapter)
    filtered_data = df_aug_train[concepts_iter][df_aug_train[concepts_iter].eq(1).any(axis=1)]
    counts.append(len(filtered_data))

# Calcul du pourcentage de lignes couvertes par rapport au total du DataFrame
percentage = [100 * c / df_aug_train.shape[0] for c in counts]

# Positions pour l'axe des x (une par concept)
positions = np.arange(len(ordered_concepts))

# --- Affichage du graphique combiné ---
fig, ax1 = plt.subplots(figsize=(18, 15))

# Graphique 1 : Barres verticales pour la valeur moyenne (ici, on peut choisir d'afficher la valeur du score associé du JSON)
# Dans cet exemple, on affiche les scores moyens provenant du JSON (extrait du plot_df précédemment)
# Si nécessaire, vous pouvez adapter la barre pour afficher une autre valeur.
values = plot_df.loc[ordered_concepts, "LIG score"]  # ou "LIG score", selon le choix

ax1.bar(positions, values, color='skyblue', alpha=0.7, label="Valeur moyenne des concepts")
ax1.set_ylabel("LIG_brute")
ax1.set_xlabel("Concepts")
ax1.set_title("Valeur moyenne des concepts et couverture cumulative")
ax1.set_xticks(positions)
ax1.set_xticklabels(ordered_concepts, rotation=45, ha='right')
ax1.grid(axis="y", linestyle="--", alpha=0.7)

# Axe secondaire : Courbe cumulative (pourcentage de lignes couvertes)
ax2 = ax1.twinx()
ax2.plot(positions, percentage, marker='o', color='darkorange', label="Couverture cumulative")
ax2.set_ylabel("Pourcentage de lignes avec au moins un concept (%)", color='darkorange')
ax2.tick_params(axis='y', colors='darkorange')
ax2.grid(axis="y", linestyle="--", alpha=0.5)

# Légendes combinées
ax1.legend(loc="upper left")
ax2.legend(loc="upper right")

plt.tight_layout()
plt.show()


In [ ]:
path_coverage = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/sorted_macro_concepts_coverage_{config.annotation}_{config.agg_mode}_{config.agg_scope}_by_lig_brute.json"
    )

with open(path_coverage, 'w') as f:
    json.dump(score_dict, f, ensure_ascii=False, indent=4)


score_dict = dict(zip(ordered_concepts, percentage))

In [ ]:
# Metrics d'aggregation par cluster

path_plot_df = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/combined_score_concept_{config.annotation}_{config.agg_mode}_{config.agg_scope}.csv"
    )

plot_df = pd.read_csv(path_plot_df)
# plot_df


In [ ]:
path_cluster_dict_strat_reassignation = f"{config.SAVE_PATH_CONCEPTS}/cluster_dict_strat_reassignation_C3M.json"

path_cluster_dict_strat_noise_cluster = f"{config.SAVE_PATH_CONCEPTS}/cluster_dict_strat_noise_cluster_C3M.json"
                                    
with open(path_cluster_dict_strat_reassignation, 'r') as f:
   cluster_dict_strat_reassignation = json.load(f)

with open(path_cluster_dict_strat_noise_cluster, 'r') as f:
    cluster_dict_strat_noise_cluster = json.load(f)

In [ ]:
def new_columns_reassignation(concept):
    for k, v in cluster_dict_strat_reassignation.items():
        if concept in v:
            return int(k)
    return -1  # ou autre valeur par défaut si pas trouvé

def new_columns_noisy(concept):
    for k, v in path_cluster_dict_strat_noise_cluster.items():
        if concept in v:
            return int(k)
    return -1  # ou autre valeur par défaut si pas trouvé

plot_df['cluster'] = plot_df['concept'].apply(new_columns_reassignation)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
import hdbscan
from umap import UMAP

# 0) On met le nom du concept en index
plot_df = plot_df.set_index("concept")

# 4) Construction : cluster → concepts triés par score
cluster_groups = {}
for c in sorted(set(set(plot_df['cluster'].to_list()))):
    lst = (plot_df
           .loc[plot_df["cluster"] == c]
           .sort_values("LIG score", ascending=False)
           .index
           .tolist())
    cluster_groups[c] = lst

# 5) Sélection round-robin + suivi du coverage par étape
total = df_aug_train.shape[0]
working = {c: cluster_groups[c].copy() for c in cluster_groups}

# Stockage des résultats
coverage_evolution = []
selected_concepts = []

while any(working[c] for c in working):
    current_selection = []
    for c in sorted(working):
        if working[c]:
            concept = working[c].pop(0)
            current_selection.append(concept)
            selected_concepts.append(concept)
    
    # Calcul du coverage après avoir ajouté ces concepts
    mask = df_aug_train[selected_concepts].eq(1).any(axis=1)
    coverage = mask.sum() * 100 / total
    
    coverage_evolution.append((current_selection.copy(), coverage))

# 6) Affichage
for idx, (concepts, cov) in enumerate(coverage_evolution):
    print(f"Step {idx+1}: Concepts {concepts} → Coverage {cov:.2f}%")



In [ ]:
import matplotlib.pyplot as plt

# Cumuler le nombre total de concepts sélectionnés à chaque étape
n_concepts_selected = np.cumsum([len(concepts) for concepts, _ in coverage_evolution])
coverage_values = [cov for _, cov in coverage_evolution]

# Tracé
plt.figure(figsize=(12, 6))
plt.plot(n_concepts_selected, coverage_values, marker='o', linestyle='-')
plt.xlabel("Nombre de concepts sélectionnés")
plt.ylabel("Coverage cumulé (%)")
plt.title("Évolution du coverage en sélection round-robin par cluster")
plt.grid(True)
plt.show()


In [ ]:
import pickle
path_coverage = (f"{config.SAVE_PATH}/blue_checkpoints/{config.model_name}/cavs"
    f"/{config.cavs_type}/sorted_macro_concepts_coverage_MJ_{config.annotation}_{config.agg_mode}_{config.agg_scope}_by_lig_brute.pkl"
    )


# Sauvegarde
with open(path_coverage, "wb") as f:
    pickle.dump(coverage_evolution, f)

print("✅ Fichier 'coverage_evolution.pkl' sauvegardé !")


In [ ]:
with open(path_coverage, "rb") as f:
    loaded_coverage_evolution = pickle.load(f)
loaded_coverage_evolution

# RANDOM ERA

In [ ]:
###### import pickle
import json
import pandas as pd
import numpy as np
from pathlib import Path

# On reprend les mêmes df et dicts que précédemment :
# - plot_df : DataFrame chargé depuis combined_score_concept_…csv
# - df_aug_train : ton DataFrame d'entraînement binaire (colonnes « concept » = 1 ou 0)
# - cluster_dict_strat_reassignation et cluster_dict_strat_noise_cluster déjà chargés

# Chemins de base
base_cavs = Path(config.SAVE_PATH) / "blue_checkpoints" / config.model_name / "cavs" / config.cavs_type

# Chargement de plot_df
path_plot_df = base_cavs / f"combined_score_concept_{config.annotation}_{config.agg_mode}_{config.agg_scope}.csv"
plot_df = pd.read_csv(path_plot_df).set_index("concept")

# Préparer les clusters
with open(f"{config.SAVE_PATH_CONCEPTS}/cluster_dict_strat_reassignation_C3M.json", "r") as f:
    cluster_reassign = json.load(f)
with open(f"{config.SAVE_PATH_CONCEPTS}/cluster_dict_strat_noise_cluster_C3M.json", "r") as f:
    cluster_noise = json.load(f)

def assign_cluster(concept, dct):
    for k,v in dct.items():
        if concept in v:
            return int(k)
    return -1

plot_df["cluster"] = plot_df.index.map(lambda c: assign_cluster(c, cluster_reassign))

# Nombre total de lignes pourcentage
total = df_aug_train.shape[0]

for run in range(10):
    # 1) Flipper / mélanger les scores LIG
    shuffled = plot_df["combined_score_LIG"].sample(frac=1, random_state=7*run+42).values
    plot_df["combined_score_LIG_shuffled"] = shuffled
    
    # 2) Construire les groupes par cluster, triés par score mélangé
    cluster_groups = {
        c: plot_df.loc[plot_df.cluster == c]
                  .sort_values("combined_score_LIG_shuffled", ascending=False)
                  .index
                  .tolist()
        for c in plot_df["cluster"].unique()
    }
    working = {c: lst.copy() for c,lst in cluster_groups.items()}

    # 3) Round‑robin et calcul du coverage cumulatif
    selected = []
    coverage_evolution = []
    while any(working[c] for c in working):
        batch = []
        for c in sorted(working):
            if working[c]:
                batch.append(working[c].pop(0))
                selected.append(batch[-1])
        # coverage après ajout de batch
        mask = df_aug_train[selected].eq(1).any(axis=1)
        coverage = mask.sum() * 100.0 / total
        coverage_evolution.append((batch.copy(), coverage))

    # 4) Sauvegarde du .pkl pour ce run
    out_path = base_cavs / f"sorted_macro_concepts_coverage_MJ_{config.annotation}_{config.agg_mode}_{config.agg_scope}_{run}.pkl"
    with open(out_path, "wb") as f:
        pickle.dump(coverage_evolution, f)
        print(coverage_evolution)
    print(f"✅ run {run} → {out_path.name} ({len(coverage_evolution)} étapes) sauvegardé.")


In [ ]:
coverage_evolution

# Other analysis : PLAYGROUND 

In [ ]:
from new_heuristique import plot_concept_threshold


In [ ]:

# On s'assure que les noms de colonnes du DataFrame sont nettoyés pour correspondre aux noms présents dans ordered_concepts
df_aug_train.rename(columns=lambda x: clean_concept_name(x), inplace=True)

# Exemple d'appel pour le concept "famous people" :
plot_concept_threshold(
    cosine_df=cosine_df, #or cosine_val pour Plotter sur le val_dataset:
    groundtruth_df=df_aug_train,  # DataFrame contenant la ground truth
    concept=cosine_df.columns.tolist()[20],
    thresholds=thresholds,
    gt_prefix="dummy_"
)

### Link between frequences and F1 score (filter method)

In [ ]:
# Compute concept presence (mean of binary values for each concept)
presence_scores = {
    concept: df_aug_train[f"dummy_{concept}"].mean()
    for concept in metrics.keys()
    if f"dummy_{concept}" in df_aug_train.columns
}

# Combine F1 and presence into a DataFrame
plot_df = pd.DataFrame({
    "F1 Score": {k: v['F1'] for k, v in metrics.items()},
    "Presence (Mean)": presence_scores
})

import matplotlib.pyplot as plt
# Plot
plt.figure(figsize=(20, 20))
plt.scatter(plot_df["Presence (Mean)"], plot_df["F1 Score"], color='dodgerblue')
for concept, row in plot_df.iterrows():
    plt.text(row["Presence (Mean)"], row["F1 Score"], concept, fontsize=9, ha='right')
plt.xlabel("Presence of Concept (Mean in Training Set)")
plt.ylabel("F1 Score (on Validation Set)")
plt.title("F1 Score vs Concept Presence")
plt.grid(True)
plt.tight_layout()
plt.show()

### Link between F1 Score (filtering methods) and F1 score (filter method)

In [ ]:
# Sauvegarder les perfs train et test des concept et aller dans le fichier new_heuristique pour voir
# le lien entre well detected et au final , bien prédit dans le CBM
with open(f"{self.config.SAVE_PATH}blue_checkpoints/{self.config.model_name}/metrics_to_compare_agnews_{config.model_name}.json", 'r') as f:
    cbm_metrics = json.load(f)
    

In [ ]:
# Combine F1 and presence into a DataFrame
plot_df_2 = pd.DataFrame({
    "F1 Score": {k: v['F1'] for k, v in metrics.items()},
    "Presence (Mean)": cbm_metrics['concept_f1_scores']
})

# Plot
plt.figure(figsize=(20, 20))
plt.scatter(plot_df_2["Presence (Mean)"], plot_df_2["F1 Score"], color='dodgerblue')
for concept, row in plot_df_2.iterrows():
    plt.text(row["Presence (Mean)"], row["F1 Score"], concept, fontsize=9, ha='right')
plt.xlabel("F1 Score on test for CBM")
plt.ylabel("F1 Score (on Validation Set) in the filtering process ")
plt.title("F1 Score (filtering process) vs F1 score (CBM)")
plt.grid(True)
plt.tight_layout()
plt.savefig("plot.png")
plt.show()


### Link between frequences and F1 score (CBM)

In [ ]:
# Compute concept presence (mean of binary values for each concept)
presence_scores = {
    concept: df_aug_train[f"dummy_{concept}"].mean()
    for concept in metrics.keys()
    if f"dummy_{concept}" in df_aug_train.columns
}

# Combine F1 and presence into a DataFrame
plot_df = pd.DataFrame({
    "F1 Score (CBM)": cbm_metrics['concept_f1_scores'],
    "Presence (Mean)": presence_scores
})

# Plot
plt.figure(figsize=(20, 20))
plt.scatter(plot_df["Presence (Mean)"], plot_df["F1 Score (CBM)"], color='dodgerblue')
for concept, row in plot_df.iterrows():
    plt.text(row["Presence (Mean)"], row["F1 Score (CBM)"], concept, fontsize=9, ha='right')
plt.xlabel("Presence of Concept (Mean in Training Set)")
plt.ylabel("F1 Score (CBM ON TEST)")
plt.title("F1 Score vs Concept Presence")
plt.grid(True)
plt.tight_layout()
plt.show()